In [1]:
import pandas as pd

pK_path = '/Users/vokeojuederhie/PycharmProjects/police_killings.csv'

# using pandas to read data
# get familiar with the data
# load the data

pK_data = pd.read_csv(pK_path, encoding='latin-1')

print(pK_data.describe())

pK_data.columns


              day    year    latitude   longitude    state_fp   county_fp  \
count  467.000000   467.0  467.000000  467.000000  467.000000  467.000000   
mean    15.830835  2015.0   36.403224  -96.972666   25.342612   91.584582   
std      8.658970     0.0    5.193357   16.953842   16.766458  110.185129   
min      1.000000  2015.0   19.915194 -159.642700    1.000000    1.000000   
25%      8.000000  2015.0   33.335240 -111.954636    8.000000   29.000000   
50%     16.000000  2015.0   35.769779  -94.761902   24.000000   63.000000   
75%     23.000000  2015.0   39.937452  -82.961582   40.000000  111.000000   
max     31.000000  2015.0   61.218408  -68.100007   56.000000  740.000000   

            tract_ce        geo_id     county_id           pop       h_income  \
count     467.000000  4.670000e+02    467.000000    467.000000     465.000000   
mean   236936.614561  2.543443e+10  25434.197002   4783.719486   46627.182796   
std    341262.721715  1.680140e+10  16801.379755   2374.565749 

Index(['name', 'age', 'gender', 'raceethnicity', 'month', 'day', 'year',
       'streetaddress', 'city', 'state', 'latitude', 'longitude', 'state_fp',
       'county_fp', 'tract_ce', 'geo_id', 'county_id', 'namelsad',
       'lawenforcementagency', 'cause', 'armed', 'pop', 'share_white',
       'share_black', 'share_hispanic', 'p_income', 'h_income',
       'county_income', 'comp_income', 'county_bucket', 'nat_bucket', 'pov',
       'urate', 'college'],
      dtype='object')

In [2]:
# Dropping missing values
pK_data = pK_data.dropna(axis=0)

y = pK_data.county_income

pK_features = ['nat_bucket','share_black', 'share_hispanic', 'share_white']

X = pK_data[pK_features]

X.describe()

,nat_bucket
count,436.000000
mean,2.502294
std,1.303398
min,1.000000
25%,1.000000
50%,2.000000
75%,4.000000
max,5.000000


In [3]:
# Model Specification

from sklearn.tree import DecisionTreeRegressor

# Define Model. Specify states to
# ensure the same result each run

pK_model = DecisionTreeRegressor(random_state=1)

# Fit Model

pK_model.fit(X,y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=1, splitter='best')

In [4]:
# Model Prediction
# listing the first few victims

print("For the first 5 victims: ")
print(X.head())

# Making predictions for these victims
print("The predictions are: ")
print(pK_model.predict(X.head()))

For the first 5 victims: 
   nat_bucket share_black share_hispanic share_white
0         3.0        30.5            5.6        60.5
1         1.0        36.2            0.5        53.8
2         3.0         7.7           16.8        73.8
3         3.0         0.6           98.8         1.2
4         4.0         1.4            1.7        92.5
The predictions are: 
[54766. 40930. 54930. 55909. 49669.]


In [5]:
# Model Validation
from sklearn.metrics import mean_absolute_error

predicted_victims = pK_model.predict(X)
mean_absolute_error(y, predicted_victims)
print(mean_absolute_error(y,predicted_victims))

0.0


In [6]:
from sklearn.model_selection import train_test_split

# splitting the data into training and validation data

train_X,val_X,train_y,val_y = train_test_split(X,y, random_state=1)

# Fit model
pK_model = DecisionTreeRegressor()
pK_model.fit(train_X, train_y)

# Get predicted victims on validation data
val_predictions = pK_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

11669.59633027523


In [7]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X,train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d \t\t Mean Absolute Error: %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5 		 Mean Absolute Error: 9259
Max leaf nodes: 50 		 Mean Absolute Error: 10385
Max leaf nodes: 500 		 Mean Absolute Error: 11847
Max leaf nodes: 5000 		 Mean Absolute Error: 11847


In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
preds = forest_model.predict(val_X)
print(mean_absolute_error(preds, val_y))



8638.100733944953
